In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [44]:
code = """select A.DRVR_USER_ID, A.MARKET_NM, 
B.ACTV_NM, B.ACTV_START_TS, B.EXPC_ACTV_END_TS, B.ACTL_ACTV_END_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM = 'Walmart Chicago' and B.ACTV_NM = "ONLINE" AND B.ACTV_START_TS BETWEEN "2022-01-01" AND "2022-07-18"
ORDER BY B.ACTV_START_TS """
df = query_bq(code)
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTL_ACTV_END_TS"] = pd.to_datetime(df["ACTL_ACTV_END_TS"])
df["ACTV_START_TS"] = df["ACTV_START_TS"] + pd.Timedelta(hours=2) #converting pacific time to eastern time (orlando market)
df["ACTL_ACTV_END_TS"] = df["ACTL_ACTV_END_TS"] + pd.Timedelta(hours=2)
df.head()

,DRVR_USER_ID,MARKET_NM,ACTV_NM,ACTV_START_TS,EXPC_ACTV_END_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,taboada4@icloud.com,Walmart Chicago,ONLINE,2022-01-01 05:45:07.381,2022-01-01 07:30:00,2022-01-01 06:22:26.588,37.320000000,2022-01-01
1,baybayturnipseed@icloud.com,Walmart Chicago,ONLINE,2022-01-01 07:05:40.896,2022-01-01 09:00:00,2022-01-01 09:08:23.100,122.720000000,2022-01-01
2,taboada4@icloud.com,Walmart Chicago,ONLINE,2022-01-01 07:56:57.280,2022-01-01 10:00:00,2022-01-01 11:55:09.370,238.200000000,2022-01-01
3,daniellebriscoe1982@gmail.com,Walmart Chicago,ONLINE,2022-01-01 09:36:37.655,2022-01-01 11:30:00,2022-01-01 10:28:45.480,52.130000000,2022-01-01
4,bisitenny@gmail.com,Walmart Chicago,ONLINE,2022-01-01 09:44:17.450,2022-01-01 10:00:00,2022-01-01 09:45:45.439,1.470000000,2022-01-01


In [19]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-11") & (df["ACTV_START_TS"] <= "2022-07-15")]["ACTV_DUR_MIN_QTY"]

0       87.420000000
1      117.380000000
2       14.430000000
3      193.920000000
4        6.870000000
           ...      
248      6.280000000
249    152.950000000
250      2.020000000
251    178.820000000
252      2.600000000
Name: ACTV_DUR_MIN_QTY, Length: 253, dtype: object

In [36]:
df = df.loc[df["ACTV_DUR_MIN_QTY"]>0]

In [30]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-11") & (df["ACTV_START_TS"] <= "2022-07-15")]["ACTV_DUR_MIN_QTY"].mean()

92.84849315068493

In [31]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-16") & (df["ACTV_START_TS"] <= "2022-07-17")]["ACTV_DUR_MIN_QTY"].mean()

79.51313725490196

In [32]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-04") & (df["ACTV_START_TS"] <= "2022-07-08")]["ACTV_DUR_MIN_QTY"].mean()

73.8709090909091

In [33]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-09") & (df["ACTV_START_TS"] <= "2022-07-10")]["ACTV_DUR_MIN_QTY"].mean()

99.47200000000001

In [37]:
df.loc[(df["ACTV_START_TS"] >= "2022-06-27") & (df["ACTV_START_TS"] <= "2022-07-01")]["ACTV_DUR_MIN_QTY"].mean()

86.65798994974874

In [38]:
df.loc[(df["ACTV_START_TS"] >= "2022-07-02") & (df["ACTV_START_TS"] <= "2022-07-03")]["ACTV_DUR_MIN_QTY"].mean()

70.2445

In [39]:
df.loc[(df["ACTV_START_TS"] >= "2022-06-20") & (df["ACTV_START_TS"] <= "2022-06-24")]["ACTV_DUR_MIN_QTY"].mean()

81.93533333333332

In [40]:
df.loc[(df["ACTV_START_TS"] >= "2022-06-25") & (df["ACTV_START_TS"] <= "2022-06-26")]["ACTV_DUR_MIN_QTY"].mean()

76.18838709677419

In [42]:
len(df.loc[df["ACTV_DUR_MIN_QTY"]<0])

245

In [48]:
df.loc[(df["ACTV_DUR_MIN_QTY"]<0)]

,DRVR_USER_ID,MARKET_NM,ACTV_NM,ACTV_START_TS,EXPC_ACTV_END_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
6056,carvilservice@gmail.com,Walmart Chicago,ONLINE,2022-03-15 04:03:43.304,2022-03-14 16:30:00,2022-03-14 18:30:00,-573.720000000,2022-03-15
6057,carvilservice@gmail.com,Walmart Chicago,ONLINE,2022-03-15 04:03:46.276,2022-03-14 16:30:00,2022-03-14 18:30:00,-573.770000000,2022-03-15
6058,nwamina.philip@gmail.com,Walmart Chicago,ONLINE,2022-03-15 04:05:03.488,2022-03-14 20:00:00,2022-03-14 22:00:00,-365.050000000,2022-03-15
6339,sparkwalmart1@gmail.com,Walmart Chicago,ONLINE,2022-03-17 12:02:31.185,2022-03-16 13:00:00,2022-03-16 15:00:00,-1262.520000000,2022-03-17
6340,sparkwalmart1@gmail.com,Walmart Chicago,ONLINE,2022-03-17 12:02:31.185,2022-03-16 13:00:00,2022-03-16 15:00:00,-1262.520000000,2022-03-17
...,...,...,...,...,...,...,...,...
14540,davidduran91@icloud.com,Walmart Chicago,ONLINE,2022-07-14 12:23:33.818,2022-05-23 11:30:00,2022-05-23 13:30:00,-74813.550000000,2022-07-14
14541,davidduran91@icloud.com,Walmart Chicago,ONLINE,2022-07-14 12:23:36.012,2022-05-23 11:30:00,2022-05-23 13:30:00,-74813.600000000,2022-07-14
14542,davidduran91@icloud.com,Walmart Chicago,ONLINE,2022-07-14 12:23:37.836,2022-05-23 11:30:00,2022-05-23 13:30:00,-74813.620000000,2022-07-14
14686,bdale83@gmail.com,Walmart Chicago,ONLINE,2022-07-17 14:39:51.908,2021-04-12 08:30:00,2021-04-12 10:30:00,-664089.850000000,2022-07-17
